In [1]:
# Install necessary packages (Uncomment if needed)
# !pip install fastapi[all] uvicorn opencv-python-headless numpy ultralytics nest_asyncio

import nest_asyncio
nest_asyncio.apply()  # Allows running asyncio loop inside Jupyter


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import json


In [ ]:
# Load YOLOv8 pose model
model = YOLO('./model/yolov8n-pose.pt')  # Use nano model for speed

# Exercise configuration
EXERCISE_CONFIG = {
    "bicep_curl": {
        "keypoints": [5, 7, 9],  # Right shoulder, elbow, wrist
        "angle_range": (30, 160),
        "rep_phase": {"up": 160, "down": 30}
    },
    "squat": {
        "keypoints": [11, 13, 15],  # Hip, knee, ankle
        "angle_range": (90, 180),
        "rep_phase": {"down": 90, "up": 160}
    }
}


In [6]:
class ExerciseProcessor:
    def __init__(self, exercise_type):
        self.config = EXERCISE_CONFIG[exercise_type]
        self.rep_count = 0
        self.current_phase = "up"
            
    def calculate_angle(self, a, b, c):
        ba = a - b
        bc = c - b
        norm_ba = np.linalg.norm(ba)
        norm_bc = np.linalg.norm(bc)
        
        if norm_ba == 0 or norm_bc == 0:
            return np.nan  # Avoid divide-by-zero
    
        cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)  # Clip to valid domain
        return np.degrees(np.arccos(cosine_angle))


    def process_frame(self, keypoints):
        points = [keypoints[i][:2] for i in self.config["keypoints"]]
        angle = self.calculate_angle(*points)
        
        form_correct = self.config["angle_range"][0] <= angle <= self.config["angle_range"][1]
        
        if angle >= self.config["rep_phase"]["up"] and self.current_phase == "down":
            self.rep_count += 1
            self.current_phase = "up"
        elif angle <= self.config["rep_phase"]["down"]:
            self.current_phase = "down"
            
        return {
            "angle": angle,
            "form_correct": form_correct,
            "rep_count": self.rep_count,
            "current_phase": self.current_phase
        }


In [8]:
import cv2
import numpy as np
from IPython.display import display, clear_output
import time

# Use OpenCV window instead of matplotlib for real-time
cap = cv2.VideoCapture(0)
processor = ExerciseProcessor("bicep_curl")

print("Press 'q' to quit.")
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (640, 480))
        results = model(resized_frame)
        keypoints = results[0].keypoints.data[0].cpu().numpy() if results[0].keypoints is not None else None

        if keypoints is not None and not np.any(np.isnan(keypoints)) and not np.any(keypoints[:, :2] == 0):
            analysis = processor.process_frame(keypoints)
            angle_display = f"{int(analysis['angle'])}°" if not np.isnan(analysis['angle']) else "N/A"
            text = f"Reps: {analysis['rep_count']} | Angle: {angle_display} | Phase: {analysis['current_phase']}"
            color = (0, 255, 0) if analysis["form_correct"] else (0, 0, 255)
            cv2.putText(resized_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            for x, y, conf in keypoints:
                cv2.circle(resized_frame, (int(x), int(y)), 5, (255, 0, 0), -1)

        # Show in OpenCV window
        cv2.imshow('Exercise Pose Detection', resized_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Stopped.")
finally:
    cap.release()
    cv2.destroyAllWindows()


Press 'q' to quit.

0: 480x640 1 person, 12.0ms
Speed: 2.3ms preprocess, 12.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 12.4ms
Speed: 1.9ms preprocess, 12.4ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 17.6ms
Speed: 2.2ms preprocess, 17.6ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.6ms
Speed: 2.0ms preprocess, 11.6ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.3ms
Speed: 1.5ms preprocess, 11.3ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.7ms
Speed: 1.7ms preprocess, 11.7ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.3ms
Speed: 1.7ms preprocess, 11.3ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.4ms
Speed: 1.7ms preprocess, 11.4ms inference, 2.0ms postprocess per image